# 참고사이트


1. detectron2 튜토리얼(풍선 인식)
https://colab.research.google.com/github/visionNoob/detectron2_aihub_tutorial/blob/master/Detectron2_Tutorial_(kor_ver).ipynb#scrollTo=EoawGUy8_33u

2. detectrton2로 너트 인식 

https://colab.research.google.com/github/Tony607/detectron2_instance_segmentation_demo/blob/master/Detectron2_custom_coco_data_segmentation.ipynb#scrollTo=PxRHYcAC_Z0f

https://www.dlology.com/blog/how-to-train-detectron2-with-custom-coco-datasets/

3. detectron2 documentation (공식 사이트?)

https://detectron2.readthedocs.io/index.html

4. detectron2에서 제공하는 여러 모델 있음. 나중에 성능비교로 써봐도 되지 않을까?? fast-rnn, faster-rnn 등

https://github.com/facebookresearch/detectron2/tree/master/configs/COCO-Detection

5. 커스텀 COCO Data Set만들기. xml -> json

https://medium.com/datadriveninvestor/how-to-create-custom-coco-data-set-for-object-detection-96ec91958f36

# 1. Detectron2 설치

In [ ]:
# 종속 패키지(dependencies) 설치
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__)

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-ga7rtlaz
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-ga7rtlaz
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-f2qh6r25
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-f2qh6r25
     |████████████████████████████████| 596 kB 4.3 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65176 sha256=32fe953370bf6d4dc5bf1b08e0dac30450a963daa44e0ed7bca799f7ad5e3c36
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wdqxe7y/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264331 sha256=3e5a6fae9decb4f1e851f05717464b9e9a1d049320b1e6e28b2d32c10bd1e980
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wdqxe7y/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a9

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
print("ok")

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 14060, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14060 (delta 4), reused 8 (delta 2), pack-reused 14046
Receiving objects: 100% (14060/14060), 5.78 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (10167/10167), done.
Obtaining file:///content/detectron2_repo
     |████████████████████████████████| 106 kB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 74 kB 3.3 MB/s 
     |████████████████████████████████| 147 kB 37.6 MB/s 
     |████████████████████████████████| 130 kB 47.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.5 MB/s 
     |████████████████████████████████| 749 kB 49.7 MB/s 
     |████████████████████████████████| 843 kB 40.2 MB/s 
     |████████████████████████████████| 112 kB 52.2 MB/s 
  Created wheel for

ok


여기까지 설치됐으면 런타임 - 런타임 다시시작으로 다시 시작한다.

In [ ]:
# 인스톨이 완료되면 Colab Runtime을 다시 시작해주세요. 

# 기본 설정
# detectron2 logger 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 자주 사용하는 라이브러리 임폴트
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# 자주 사용하는 detectron2 유틸 임폴트 
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

print("ok")

ok


# 2. 데이터셋 가져오기

깃허브 레지트리에 넣어두었다. 이미지와 xml파일을 json으로 바꾼 것이 있다. 

In [ ]:
# 데이터를 불러와서 압축을 해제합니다.
!wget https://github.com/mulkkog/Waterdeer/releases/download/1.0/waterdeer_dataset.zip
!unzip waterdeer_dataset.zip > /dev/null
print("ok")

--2022-05-14 12:54:46--  https://github.com/mulkkog/Waterdeer/releases/download/1.0/waterdeer_dataset.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-14 12:54:46 ERROR 404: Not Found.

unzip:  cannot find or open waterdeer_dataset.zip, waterdeer_dataset.zip.zip or waterdeer_dataset.zip.ZIP.
ok


"고라니 데이터셋"을 detectron2에 등록시킨다. 

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_waterdeer_train", {}, "./waterdeer/train/via_region_data.json", "./waterdeer/train")
register_coco_instances("my_waterdeer_val", {}, "./waterdeer/val/via_region_data.json", "./waterdeer/val")

In [ ]:
waterdeer_metadata = MetadataCatalog.get("my_waterdeer_train")
print(waterdeer_metadata)

Metadata(evaluator_type='coco', image_root='./waterdeer/train', json_file='./waterdeer/train/via_region_data.json', name='my_waterdeer_train')


데이터가 제대로 로드되었는지 확인. 랜덤으로 3개

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_dicts = DatasetCatalog.get("my_waterdeer_train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=waterdeer_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

FileNotFoundError: ignored

#3. Object Detection

학습하기. COCO데이터셋으로 학습된 R50-FPN RetinaNet 모델

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

#create config
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_waterdeer_train",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/retinanet_R_50_FPN_3x/137849486/model_final_4cafe0.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.RETINANET.NUM_CLASSES = 1 # 1 classes (waterdeer)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Loading config ./detectron2_repo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[05/14 12:55:03 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

FileNotFoundError: ignored

텐서보드로 학습 과정 시각화

In [ ]:
# tensorboard를 사용해서 학습 커브를 살펴봅니다.
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("my_waterdeer_val", )
predictor = DefaultPredictor(cfg)
print(cfg.MODEL.RETINANET.NUM_CLASSES)

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=waterdeer_metadata, 
                   scale=0.5, 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

학습한 모델 실행 및 평가

inference speed(추론 스피드) 

In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_waterdeer_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_waterdeer_val")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test